# EDA - Fraud E-commerce Data

## Task 1: Data Analysis and Preprocessing

**Objective**: Prepare clean, feature-rich datasets ready for modeling by exploring the data, engineering meaningful features, and handling class imbalance.

This notebook focuses on:
- Data Cleaning (missing values, duplicates, data types)
- Exploratory Data Analysis
- Feature Engineering
- Handling Class Imbalance

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

# Set style for plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")


Libraries imported successfully!


## 1. Load the Data


In [2]:
# Define paths
data_dir = Path('../data')
raw_data_path = data_dir / 'raw' / 'Fraud_Data.csv'
processed_data_path = data_dir / 'processed'

# Create processed directory if it doesn't exist
processed_data_path.mkdir(parents=True, exist_ok=True)

# Load the dataset
print("Loading fraud e-commerce dataset...")
df = pd.read_csv(raw_data_path)

print(f"\nDataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()


Loading fraud e-commerce dataset...

Dataset loaded successfully!
Shape: (151112, 11)

First few rows:


,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


## 2. Initial Data Exploration


In [3]:
# Basic information about the dataset
print("=" * 120)
print("DATASET INFORMATION")
print("=" * 120)
print(f"\nShape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"\nColumn Names:")
print(df.columns.tolist())
print(f"\nData Types:")
print(df.dtypes)
print(f"\nDataset Info:")
df.info()


DATASET INFORMATION

Shape: 151112 rows × 11 columns

Column Names:
['user_id', 'signup_time', 'purchase_time', 'purchase_value', 'device_id', 'source', 'browser', 'sex', 'age', 'ip_address', 'class']

Data Types:
user_id             int64
signup_time        object
purchase_time      object
purchase_value      int64
device_id          object
source             object
browser            object
sex                object
age                 int64
ip_address        float64
class               int64
dtype: object

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         151112 non-null  int64  
 1   signup_time     151112 non-null  object 
 2   purchase_time   151112 non-null  object 
 3   purchase_value  151112 non-null  int64  
 4   device_id       151112 non-null  object 
 5   source          151112 non-null  

In [4]:
# Statistical summary
print("=" * 120)
print("STATISTICAL SUMMARY")
print("=" * 120)
df.describe(include='all')


STATISTICAL SUMMARY


,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
count,151112.000000,151112,151112,151112.000000,151112,151112,151112,151112,151112.000000,1.511120e+05,151112.000000
unique,NaN,151112,150679,NaN,137956,3,5,2,NaN,NaN,NaN
top,NaN,2015-02-24 22:55:49,2015-07-17 23:22:55,NaN,NGQCKIADMZORL,SEO,Chrome,M,NaN,NaN,NaN
freq,NaN,1,3,NaN,20,60615,61432,88293,NaN,NaN,NaN
mean,200171.040970,NaN,NaN,36.935372,NaN,NaN,NaN,NaN,33.140704,2.152145e+09,0.093646
std,115369.285024,NaN,NaN,18.322762,NaN,NaN,NaN,NaN,8.617733,1.248497e+09,0.291336
min,2.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,18.000000,5.209350e+04,0.000000
25%,100642.500000,NaN,NaN,22.000000,NaN,NaN,NaN,NaN,27.000000,1.085934e+09,0.000000
50%,199958.000000,NaN,NaN,35.000000,NaN,NaN,NaN,NaN,33.000000,2.154770e+09,0.000000
75%,300054.000000,NaN,NaN,49.000000,NaN,NaN,NaN,NaN,39.000000,3.243258e+09,0.000000


## 3. Data Cleaning

### 3.1 Check for Missing Values


In [6]:
# Check for missing values
print("=" * 120)
print("MISSING VALUES ANALYSIS")
print("=" * 120)

missing_count = df.isnull().sum()
missing_percent = (missing_count / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing_count.index,
    'Missing Count': missing_count.values,
    'Missing Percentage': missing_percent.values
})

missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

if len(missing_df) > 0:
    print("\nColumns with missing values:")
    print(missing_df)
else:
    print("\n✓ No missing values found in the dataset!")

# Visualize missing values if any exist
if len(missing_df) > 0:
    plt.figure(figsize=(10, 6))
    sns.barplot(data=missing_df, x='Column', y='Missing Percentage')
    plt.title('Missing Values by Column')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()


MISSING VALUES ANALYSIS

✓ No missing values found in the dataset!


### 3.2 Handle Missing Values

**Strategy**: 
- For numerical columns: Use median imputation (robust to outliers)
- For categorical columns: Use mode imputation or mark as 'Unknown'
- For datetime columns: Drop rows if critical, otherwise forward fill
- **Justification**: We'll handle missing values based on the column type and business logic


In [7]:
# Store original shape
original_shape = df.shape
print(f"Original dataset shape: {original_shape}")

# Handle missing values based on column type
if df.isnull().sum().sum() > 0:
    print("\nHandling missing values...")
    
    # Numerical columns - use median imputation
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    for col in numerical_cols:
        if df[col].isnull().sum() > 0:
            median_val = df[col].median()
            df[col].fillna(median_val, inplace=True)
            print(f"  - Filled {col} with median: {median_val}")
    
    # Categorical columns - use mode imputation
    categorical_cols = df.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        if df[col].isnull().sum() > 0:
            mode_val = df[col].mode()[0] if len(df[col].mode()) > 0 else 'Unknown'
            df[col].fillna(mode_val, inplace=True)
            print(f"  - Filled {col} with mode: {mode_val}")
    
    # Verify no missing values remain
    remaining_missing = df.isnull().sum().sum()
    print(f"\n✓ Missing values handled. Remaining missing: {remaining_missing}")
else:
    print("\n✓ No missing values to handle!")


Original dataset shape: (151112, 11)

✓ No missing values to handle!


### 3.3 Remove Duplicates


In [9]:
# Check for duplicate rows
print("=" * 120)
print("DUPLICATE ROWS ANALYSIS")
print("=" * 120)

duplicate_count = df.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicate_count}")

if duplicate_count > 0:
    duplicate_percent = (duplicate_count / len(df)) * 100
    print(f"Percentage of duplicates: {duplicate_percent:.2f}%")
    
    # Show some duplicate examples
    print("\nSample duplicate rows:")
    print(df[df.duplicated(keep=False)].head(10))
    
    # Remove duplicates
    print(f"\nRemoving {duplicate_count} duplicate rows...")
    df = df.drop_duplicates()
    print(f"✓ Duplicates removed. New shape: {df.shape}")
else:
    print("\n✓ No duplicate rows found!")


DUPLICATE ROWS ANALYSIS

Number of duplicate rows: 0

✓ No duplicate rows found!


### 3.4 Correct Data Types

**Strategy**:
- Convert `signup_time` and `purchase_time` to datetime
- Ensure `user_id` is integer
- Ensure `purchase_value` is numeric
- Ensure `age` is integer
- Ensure `class` is integer (binary: 0 or 1)
- Convert categorical columns to appropriate types


In [10]:
# Store data types before conversion
print("=" * 120)
print("DATA TYPE CORRECTION")
print("=" * 120)
print("\nOriginal data types:")
print(df.dtypes)


DATA TYPE CORRECTION

Original data types:
user_id             int64
signup_time        object
purchase_time      object
purchase_value      int64
device_id          object
source             object
browser            object
sex                object
age                 int64
ip_address        float64
class               int64
dtype: object


In [11]:
# Convert datetime columns
print("\nConverting datetime columns...")
df['signup_time'] = pd.to_datetime(df['signup_time'], errors='coerce')
df['purchase_time'] = pd.to_datetime(df['purchase_time'], errors='coerce')

# Check for any conversion errors
datetime_errors = df[['signup_time', 'purchase_time']].isnull().sum()
if datetime_errors.sum() > 0:
    print(f"  ⚠ Warning: {datetime_errors.sum()} rows have invalid datetime values")
    # Drop rows with invalid datetime if critical
    if datetime_errors.sum() < len(df) * 0.01:  # Less than 1% of data
        df = df.dropna(subset=['signup_time', 'purchase_time'])
        print(f"  - Dropped {datetime_errors.sum()} rows with invalid datetime")
    else:
        print(f"  - Keeping rows, will handle in feature engineering")
else:
    print("  ✓ Datetime conversion successful!")

print(f"\nAfter datetime conversion shape: {df.shape}")



Converting datetime columns...
  ✓ Datetime conversion successful!

After datetime conversion shape: (151112, 11)


In [12]:
# Convert numeric columns to appropriate types
print("\nConverting numeric columns...")

# User ID - should be integer
if df['user_id'].dtype != 'int64':
    df['user_id'] = pd.to_numeric(df['user_id'], errors='coerce').astype('Int64')
    print("  ✓ user_id converted to integer")

# Purchase value - should be numeric
if df['purchase_value'].dtype not in ['int64', 'float64']:
    df['purchase_value'] = pd.to_numeric(df['purchase_value'], errors='coerce')
    print("  ✓ purchase_value converted to numeric")

# Age - should be integer
if df['age'].dtype != 'int64':
    df['age'] = pd.to_numeric(df['age'], errors='coerce').astype('Int64')
    print("  ✓ age converted to integer")

# IP address - keep as float (it's already numeric)
print("  ✓ ip_address kept as float")

# Class - should be integer (binary: 0 or 1)
if df['class'].dtype != 'int64':
    df['class'] = pd.to_numeric(df['class'], errors='coerce').astype('int64')
    print("  ✓ class converted to integer")



Converting numeric columns...
  ✓ ip_address kept as float


In [15]:
# Convert categorical columns
print("\nConverting categorical columns...")

categorical_columns = ['source', 'browser', 'sex', 'device_id']

for col in categorical_columns:
    if col in df.columns:
        df[col] = df[col].astype('category')
        print(f"  ✓ {col} converted to category")

print("\n" + "=" * 120)
print("FINAL DATA TYPES")
print("=" * 120)
print(df.dtypes)



Converting categorical columns...
  ✓ source converted to category
  ✓ browser converted to category
  ✓ sex converted to category
  ✓ device_id converted to category

FINAL DATA TYPES
user_id                    int64
signup_time       datetime64[ns]
purchase_time     datetime64[ns]
purchase_value             int64
device_id               category
source                  category
browser                 category
sex                     category
age                        int64
ip_address               float64
class                      int64
dtype: object


### 3.5 Data Cleaning Summary


In [16]:
# Summary of data cleaning
print("=" * 120)
print("DATA CLEANING SUMMARY")
print("=" * 120)
print(f"\nOriginal dataset shape: {original_shape}")
print(f"Final dataset shape: {df.shape}")
print(f"Rows removed: {original_shape[0] - df.shape[0]}")
print(f"Columns: {df.shape[1]}")

print(f"\n✓ Missing values: {df.isnull().sum().sum()}")
print(f"✓ Duplicate rows: {df.duplicated().sum()}")
print(f"\n✓ Data types corrected:")
print(f"  - Datetime: signup_time, purchase_time")
print(f"  - Integer: user_id, age, class")
print(f"  - Numeric: purchase_value, ip_address")
print(f"  - Categorical: source, browser, sex, device_id")

print(f"\n✓ Dataset is ready for further analysis!")
df.head()


DATA CLEANING SUMMARY

Original dataset shape: (151112, 11)
Final dataset shape: (151112, 11)
Rows removed: 0
Columns: 11

✓ Missing values: 0
✓ Duplicate rows: 0

✓ Data types corrected:
  - Datetime: signup_time, purchase_time
  - Integer: user_id, age, class
  - Numeric: purchase_value, ip_address
  - Categorical: source, browser, sex, device_id

✓ Dataset is ready for further analysis!


,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


## 4. Save Cleaned Dataset


In [17]:
# Save cleaned dataset
cleaned_file_path = processed_data_path / 'fraud_data_cleaned.csv'
df.to_csv(cleaned_file_path, index=False)
print(f"✓ Cleaned dataset saved to: {cleaned_file_path}")
print(f"  Shape: {df.shape}")
print(f"  File size: {cleaned_file_path.stat().st_size / (1024*1024):.2f} MB")


✓ Cleaned dataset saved to: ../data/processed/fraud_data_cleaned.csv
  Shape: (151112, 11)
  File size: 14.18 MB
